In [2]:
!pip install opendatasets
import opendatasets as od

In [3]:
od.download('https://www.kaggle.com/datasets/adilshaik6571/ecg-dataset')

{"username":"adilshaik6571","key":"ffe6fa7417b6418837e963b547f9012e"}

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/adilshaik6571/ecg-dataset


{'username': 'adilshaik6571', 'key': 'ffe6fa7417b6418837e963b547f9012e'}

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
import numpy as np


In [26]:
# Set your data directory
data_directory = "ecg-dataset/ECG_Image_data/train"


In [27]:
# Define image size and batch size
img_size = (128, 128)
batch_size = 32

In [28]:
# Create data generators for training, validation, and testing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',  # Use grayscale images
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False  # Don't shuffle for evaluation
)

Found 72638 images belonging to 5 classes.
Found 18156 images belonging to 5 classes.
Found 90794 images belonging to 5 classes.


In [29]:
# Create the LeNet-5 model
model = models.Sequential([
    layers.Conv2D(6, (5, 5), activation='relu', input_shape=(128, 128, 1)),  # Use 'grayscale' images, so input_shape=(128, 128, 1)
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(5, activation='softmax')  # 5 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\ADIL DRAGSTER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [30]:
print("Classes found:", train_generator.class_indices)
print("Total classes:", train_generator.num_classes)

Classes found: {'F': 0, 'N': 1, 'Q': 2, 'S': 3, 'V': 4}
Total classes: 5


In [31]:
# Train the model

epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10


c:\Users\ADIL DRAGSTER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2270/2270 ━━━━━━━━━━━━━━━━━━━━ 0s 622ms/step - accuracy: 0.9425 - loss: 0.2046

KeyboardInterrupt: 

In [32]:
model.save('LeNet_model.h5')

In [33]:
# Evaluate the model on the test set
test_results = model.evaluate(test_generator)

# Print test metrics
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

2838/2838 ━━━━━━━━━━━━━━━━━━━━ 410s 144ms/step - accuracy: 0.9987 - loss: 0.0039
Test Loss: 0.011385872960090637
Test Accuracy: 0.9959468841552734


In [34]:
# Generate predictions on the test set
predictions = model.predict(test_generator)

2838/2838 ━━━━━━━━━━━━━━━━━━━━ 395s 139ms/step


In [35]:
# Convert predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = test_generator.classes

In [36]:
# Print classification report and confusion matrix
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

           F       1.00      0.98      0.99       642
           N       1.00      1.00      1.00     75709
           Q       1.00      1.00      1.00      6431
           S       0.89      0.96      0.93      2223
           V       0.98      0.96      0.97      5789

    accuracy                           1.00     90794
   macro avg       0.98      0.98      0.98     90794
weighted avg       1.00      1.00      1.00     90794


Confusion Matrix:
[[  627     0     0     5    10]
 [    0 75706     2     0     1]
 [    0    11  6420     0     0]
 [    0     0     0  2132    91]
 [    0     0     0   248  5541]]


In [42]:
# Test an individual image
def test_single_image(image_path):
    img = image.load_img(image_path, target_size=(128, 128), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    predicted_label = class_labels[predicted_class]
    print(f"Predicted class: {predicted_label}")


# Example usage for testing a single image
test_single_image(r"D:\Major Project\Main_Project\ecg-dataset\ECG_Image_data\test\V\V1104.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
Predicted class: V
